In [1]:
import tensorflow as tf
#import tensorflowjs as tfjs
from matplotlib import pyplot
import numpy as np
import os
import json

ModuleNotFoundError: No module named 'tensorflowjs'

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory("data-150x150",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(150, 150),
  batch_size=32)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory("data-150x150",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(150, 150),
  batch_size=32)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal", input_shape=(150,150,3)),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomCrop(120, 120) #height, width
  ]
)

In [ ]:
num_classes = 4
models = {}

In [ ]:
models["CNN"] = tf.keras.Sequential([
  #data_augmentation,
  tf.keras.layers.RandomFlip("horizontal", input_shape=(150,150,3)),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2),
  tf.keras.layers.RandomContrast(0.2),
  tf.keras.layers.RandomCrop(120, 120),
  tf.keras.layers.Rescaling(1./255, input_shape=(150, 150, 3)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
models["MLP"] = tf.keras.Sequential([
    data_augmentation,
    tf.keras.layers.Flatten(input_shape=(150, 150, 3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

In [ ]:
for model in models.values():
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
for model in models.values():
    model.summary()

In [ ]:
epochs=2
histories = {}

In [ ]:
for name in models:
    print("Training "+name+":")
    model_path = os.path.join("saved_models", name+".h5")
    history_path = os.path.join("saved_histories", name+".json")
    #tfjs_target_path = os.path.join("web-client", "saved_models", name)
    histories[name] = models[name].fit(
      train_ds,
      validation_data=val_ds,
      epochs=epochs,
    )
    models[name].save(model_path)
    #tfjs.converters.save_keras_model(model[name], tfjs_target_path)
    json.dump(histories[name].history, open(history_path, 'w'))
    print()

In [ ]:
for name in histories:
    history = histories[name]
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    pyplot.figure(figsize=(8, 8))
    pyplot.subplot(1, 2, 1)
    pyplot.plot(epochs_range, acc, label='Training Accuracy')
    pyplot.plot(epochs_range, val_acc, label='Validation Accuracy')
    pyplot.legend(loc='lower right')
    pyplot.title(name+': Training and Validation Accuracy')

    pyplot.subplot(1, 2, 2)
    pyplot.plot(epochs_range, loss, label='Training Loss')
    pyplot.plot(epochs_range, val_loss, label='Validation Loss')
    pyplot.legend(loc='upper right')
    pyplot.title(name+': Training and Validation Loss')
    pyplot.show()

In [ ]:
class_names = ["Wearing Mask a Correctly Over Mouth and Nose", "Wearing Mask a Over Mouth, but not Over Nose", "Not Wearing a Mask Over Mount and Nose", "Not Wearing a Mask"]

In [ ]:
def predict(file_path):
    
    img = tf.keras.utils.load_img(file_path, target_size=(150, 150))
    pyplot.imshow(img)

    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    
    for name in models:
        model = models[name]
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])


        print(
            "{} predicts {} ({:.2f}% confidence)"
            .format(name, class_names[np.argmax(score)], 100 * np.max(score))
        )
    pyplot.show()

In [ ]:
predict("faces/benjamin/1.jpg")

In [ ]:
predict("faces/benjamin/2.jpg")

In [ ]:
predict("faces/benjamin/3.jpg")

In [ ]:
predict("faces/benjamin/4.jpg")

In [ ]:
predict("faces/shaurya/1.jpg")

In [ ]:
predict("faces/shaurya/2.jpg")

In [ ]:
predict("faces/shaurya/3.jpg")

In [ ]:
predict("faces/shaurya/4.jpg")

In [ ]:
predict("faces/zunyi/1.jpg")

In [ ]:
predict("faces/zunyi/2.jpg")

In [ ]:
predict("faces/zunyi/3.jpg")

In [ ]:
predict("faces/zunyi/4.jpg")

In [ ]:
predict("faces/parisa/1.jpg")

In [ ]:
predict("faces/parisa/2.jpg")

In [ ]:
predict("faces/parisa/3.jpg")

## Source: https://www.tensorflow.org/tutorials/images/classification